# Drop and Pop

#### 1. Notebook will get the list of stocks from the csv file

In [1]:
import csv

STOCKS_FILE_NAME = 'DropAndPopQuotes.csv'

def get_stocks():
    with open(STOCKS_FILE_NAME, 'r') as f:
        reader = csv.reader(f, delimiter=',', quotechar='"')
        stocks = [stock.strip() for row in reader for stock in row]
    return stocks

stocks = get_stocks()
stocks[:5]

['AAPL', 'NFLX', 'BAC', 'NVDA', 'LUV']

#### 2. Notebook will get yesterday’s close price and current trading price

In [3]:
import pandas as pd
import requests
df = pd.DataFrame(columns=('Open', 'PrevClose', 'OpenByClose'))
for stock in get_stocks():
    url = "https://query1.finance.yahoo.com/v7/finance/options/{}".format(stock)
    page = requests.get(url)
    response = page.json()    
    results = response['optionChain']['result']
    for result in results:
        quote = result['quote']
        open_price = quote['regularMarketOpen']
        previous_close = quote['regularMarketPreviousClose']
        open_by_close = (open_price / previous_close - 1) * 100
        df.loc[stock] = open_price, previous_close, open_by_close

df

,Open,PrevClose,OpenByClose
AAPL,127.99000,128.75000,-0.590291
NFLX,140.61000,140.78000,-0.120756
BAC,22.74000,22.89000,-0.655308
NVDA,113.31000,113.95000,-0.561650
LUV,52.27000,52.73000,-0.872369
FB,133.23000,133.23000,0.000000
AMZN,836.59000,832.35004,0.509396
CVS,76.35000,77.97000,-2.077722
GALE,1.29000,1.28000,0.781250
PULM,3.73000,3.73000,0.000000


#### 3. Stocks that gapped down by at least -5% will be on the ‘follow list’

In [4]:
follow_list = df[df.OpenByClose <= -5]
follow_list

,Open,PrevClose,OpenByClose


#### 4 and 5. For these stocks, the Notebook starts to track options . The table will update on a minute basis and will show our standard option table PLUS 2 more columns: (Open / Close drop (e.g., -7%), Current / Open (e.g., +2%))

In [5]:
import queue
import requests
import threading


def load_data(stocks, time):
    
    num_worker_threads = 100
    
    def worker():
        while True:
            args = q.get()
            if args is None:
                break
            get_transactions(q, r, *args)  # <-- see definition of this function below
            q.task_done()

    q = queue.Queue() # queue with arguments for the 'get_transactions' function
    r = queue.Queue() # queue with raw responses

    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    for stock in stocks:
        q.put((stock,))

    # block until all tasks are done
    q.join()

    # stop workers
    for i in range(num_worker_threads):
        q.put(None)

    for t in threads:
        t.join()
        
    save_rows(r, time)    
    
def get_transactions(queue, resp_queue, stock, date=None):     
    base_url = "https://query1.finance.yahoo.com/v7/finance/options/{}".format(stock)
    if date is not None:
        url = "{}?date={}".format(base_url, date)
    else:
        url = base_url
    try:
        page = requests.get(url)
    except Exception as e:
        print(e)
    else:
        response = page.json()
        if response['optionChain']['error']: # if any error
            print(response['optionChain']['error'])

        else:            
            results = response['optionChain']['result']
            for result in results:
                quote = result['quote']
                open_price = quote['regularMarketOpen']
                previous_close = quote['regularMarketPreviousClose']
                open_by_close = (open_price / previous_close - 1) * 100
                if open_by_close <= PRICE_DROP_PERCENT:                    
                    price = quote['regularMarketPrice']
                    current_by_open = (price / open_price - 1) * 100
                    for option in result['options']:
                        resp_queue.put(
                            (stock, price, open_by_close, current_by_open, option['calls'], option['puts'])
                        )  
                    if date is None:
                        for tm in result['expirationDates'][1:]:
                            queue.put((stock, tm))
                        
def save_rows(q, time):
    while True:
        try:
            stock, price, open_by_close, current_by_open, calls, puts = q.get(block=False)
        except queue.Empty:
            break
        else:
            if stock not in follow_stocks:
                follow_stocks.append(stock)
                
            for call in calls:
                save_row(stock, price, open_by_close, current_by_open, 'Calls', time, call)
            for put in puts:
                save_row(stock, price, open_by_close, current_by_open, 'Puts', time, put)

                
def save_row(s, price, open_by_close, current_by_open, t, time, raw):
    contact = raw['contractSymbol']    
    previous_volume = transaction_volumes.get(contact)
    volume = raw['volume']
    if previous_volume is None:
        transaction_volumes[contact] = volume
    else:         
        volume_diff = volume - previous_volume
        if volume_diff > 0:            
            transaction_volumes[contact] = volume
            
            last_price = raw['lastPrice']
            strike = raw['strike']
            ask = raw['ask']
            bid = raw['bid']
            if s not in largest_transactions:
                largest_transactions[s] = []
            largest_transactions[s].append(
                dict(
                    stock=s,
                    price=price,
                    type=t,
                    open_by_close=open_by_close,
                    current_by_open=current_by_open,
                    time=time,
                    expiration=dt.fromtimestamp(raw['expiration']),
                    strike=strike,
                    contact=contact,
                    last_price=last_price,
                    bid=bid,
                    ask=ask,
                    volume=volume,
                    open_interest=raw['openInterest'],
                    implied_volatility=raw['impliedVolatility'],
                    volume_diff=volume_diff,
                    cost=volume_diff * 100 * last_price,
                    strike_div_price=(strike / price - 1) * 100,
                    last_price_position="{:.0f}".format((1 - (ask-last_price) / (ask-bid)) * 100) if ask != bid else None
                )
            )

In [6]:
from IPython.display import clear_output, display, HTML

headers = (
    "Stock", "Share $", "Options", "Volume", "Open Interest", "Strike", "S / S", 
    "Expiration", "Last price", "Bid", "Ask", "Position",  "Value", "Time of trade", 
    "Open/Close", "Price/Open",
)
row = """
<td>{stock}</td><td>{price}</td><td>{type}</td><td>{volume_diff}</td><td>{open_interest}</td>
<td>{strike}</td><td>{strike_div_price:.2f}%</td><td>{expiration:%b %d, %Y}</td>
<td>{last_price}</td><td>{bid}</td><td>{ask}</td><td>{last_price_position}%</td>
<td>${cost:,.0f}</td><td>{time:%I:%M%p}</td><td>{open_by_close:.0f}%</td><td>{current_by_open:.0f}%</td>
"""

def display_data():    
    table_rows = (row.format(**row_data) for rows in largest_transactions.values() for row_data in rows)           
    clear_output()
    display(
        HTML(
            '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format(
                "</th><th>".join(headers),
                "</tr><tr>".join(table_rows)
            )
        )
    ) 

In [8]:
from datetime import datetime as dt, timedelta as td
from collections import OrderedDict
from pytz import timezone
from time import sleep
import heapq

tz = timezone('America/Los_Angeles')
STOCKS_FILE_NAME = 'DropAndPopQuotes.csv'
PRICE_DROP_PERCENT = -5 
TOP_ITEMS_COUNT = 20
all_stocks = get_stocks()
follow_stocks = []

last_time = None
count = 1
largest_transactions = {}  # here we will store top entries
transaction_volumes = {}  # latest volumes for transactions

In [ ]:
# The main proccess loop
while True:    
    now = dt.now(tz=tz)
    weekday = now.isoweekday()
    
    # check stock symbols to follow
    if last_time is None or last_time.day != now.day:
        follow_stocks = []
    
    start = now.replace(hour=6, minute=30, second=0, microsecond=0)
    stop = start.replace(hour=13, minute=0)
    if weekday < 6 and start <= now <= stop: # proper time and day for getting data
        
        if last_time: # no more often than once per minute
            diff = 60 - (now - last_time).seconds            
            if diff > 0:
                sleep(diff)         
        
        load_data(follow_stocks or all_stocks, now)
        
        if not follow_stocks:
            print("The follow list is empty")
            sleep(60 * 60) # sleep for an hour
            continue
        
        # get top transactions
        for k, v in largest_transactions.items():
            largest_transactions[k] = heapq.nlargest(TOP_ITEMS_COUNT, v, key=lambda i: i['cost'])
            
        # sort stocks by cost sum
        largest_transactions = OrderedDict(
            sorted(largest_transactions.items(), key=lambda t: sum(i['cost'] for i in t[1]), reverse=True)
        )        
        display_data()  
        
        # some statistics
        print("prev update", last_time)
        print("last update", now)         
        print("update time", dt.now(tz=tz) - now)
        print("iterations", count)  
        print(follow_stocks)
        count += 1
        last_time = now 
    else:
        
        if weekday < 6 and now < start:
            diff = start - now            
        else:
            largest_transactions = {} # drop transactions
            
            days = 8 - weekday if weekday > 5 else 1                
            next_start = start + td(days=days)
            diff = next_start - now
            
        print("time to next start:", diff)
        sleep(diff.seconds + diff.microseconds * 0.1 ** 6)

The follow list is empty
